In [1]:
import re
import os
import numpy as np
from random import randint

from gensim.models import Word2Vec

In [10]:
n_train = 500_000
DATA_FILE = os.path.join("data", "notex_all.csv")
WORD_PATTERN = re.compile('\w[\w\'`]+')
# GENSIM_WORD_VEC_FILE = os.path.join("gensim", f"w2v_{n_train//1000}k_notex_records.kv")
EMBEDD_FILE = os.path.join("gensim", "embedd_weights.npy")
EMBEDD_DIM = 300
X_FILE = os.path.join("gensim", "embedded_X.txt")
Y_FILE = os.path.join("gensim", "encoded_Y.txt")

In [25]:
# A python iterator for gensim's Word2Vec class
# It spits out elements that are lists of words
# Each list corresponds to one arXiv article
# e.g. next(texts_iter) = ['hello', 'world']

class texts_iter():
    
    def __init__(self, filename, nrows=float('inf'), skiprows=1,\
                 prepend_label=True):
        
        self.idx = 0
        self.nrows = nrows
        self.records = open(filename, "r")
        self.maxlen = 0
        self.prepend_label = prepend_label
        
        #skip rows, default = 1-line header
        for _ in range(skiprows):
            next(self.records)
    
    def finish(self):
        self.records.close()


    # the defining method, returns a list
    def __next__(self):
        
        # don't read beyond 'nrows'
        if self.idx < self.nrows:  
            self.idx += 1
            
            try:
                record, label = next(self.records).split('\t')
            except:
                self.finish()
                raise StopIteration()
            
                
            text = record.lower()
            words = WORD_PATTERN.findall(text)
            self.maxlen = max(len(words), self.maxlen)
            
            if self.prepend_label:
                return [label.strip()] + words
            else:
                return words
        
        else:
            self.finish()
            raise StopIteration()
    
    def __iter__(self):
        return self

In [7]:
# apply Word2Vec
texts_train = texts_iter(DATA_FILE, nrows=n_train, prepend_label=False)
word2vec = Word2Vec(texts_train,\
                    size=EMBEDD_DIM,\
                    min_count=1, sorted_vocab=1).wv

print(f"Highest wordcount in the train data-set: {texts_train.maxlen}.")
# word2vec.save(GENSIM_WORD_VEC_FILE)

Highest wordcount in the train data-set: 647.


In [24]:
PADDED_LEN = int(1.05 * 647)
PADDED_LEN

679

In [15]:
# # load the gensim's "wv" object from the file
# word2vec = KeyedVectors.load(WORD_VEC_TRAIN_FILE, mmap='r')

In [17]:
# The resulting embedding and the mapping from words to indeces
# are contained in this wod2vec object
# Note that one needs the map along with the embedding to properly
# tokenize the texts

embedding = np.asarray(word2vec.vectors)
print(embedding.shape)

def word_to_idx(word):
    try:
        return word2vec.vocab[word].index + 1
    except:
        return 0

(299304, 300)


In [18]:
word_to_idx('the'), word_to_idx('świerszcz')

(1, 0)

In [19]:
# save the embedding weights to file
np.save(EMBEDD_FILE, embedding)

In [26]:
# create new files:
# one with the texts converted to sequences of integer tokens (0-padded)
# and the second with integer-encoded labels,
# both using the texts_iter class
# Note that the tokenization is applied to the whole of data
# including test-data (so don't look inside)

label_encoding = {}

with open(X_FILE, 'w') as x_file:
    with open(Y_FILE, 'w') as y_file:

        # generate word-lists using the same iterator as for
        # creating the embedding
        records = texts_iter(DATA_FILE, prepend_label=True)

        for list_ in records:
            
            label, *text = list_
            
            if label in label_encoding:
                y = label_encoding[label.strip()]
            else:
                y = len(label_encoding)
                label_encoding[label] = y
                
            y_file.write(str(y) + '\n')
            
            sequence = [str(word_to_idx(word)) for word in text]
            
            # add post-padding
            sequence += [str(0)] * (PADDED_LEN - len(sequence))
            sequence = sequence[:PADDED_LEN]
            
            x_file.write(" ".join(sequence) + '\n')
            
label_encoding

{'phys': 0, 'math': 1, 'q-bio': 2, 'stat': 3, 'q-fin': 4, 'cs': 5}

In [ ]:
# # suboptimal function to randomly insert zeros into a list
# # the number of zeros is proportional to the length of the list

# def sprinkle_zeros(list_, frac):
#     k = int(frac * len(list_))
#     for _ in range(k):
#         list_.insert(randint(0,len(list_)), 0)

# li = list(range(1,20))
# sprinkle_zeros(li, 0.1)
# li